In [1]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from nltk.tokenize import sent_tokenize
import unicodedata
from elasticsearch import helpers
from datetime import datetime
import glob
import ast
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet 
from collections import Counter
import spacy
from nltk import ne_chunk,pos_tag
from nltk.tree import Tree
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

### Document-Term Matrix
Previously method used in hw3 is not doable because now we are stripping each news as an item instead of each sentences. 

Don't run it over and over again after initial run- not necessary

In [105]:
path='C:\\Users\\Amy\\Downloads\\all1314text\\*.txt'
files=glob.glob(path)
text=''
news=[]
for document in files:
    with open(document,'r',errors='ignore') as single_file:
        articles=[]
        for eachline in single_file:
            #eachline is a sentence
            newsart = unicodedata.normalize('NFKD', eachline).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            articles.append(newsart)
    news.append(articles)
    if i%50==0:
        print (f"Read {document.name}")
    
    
    # Write to a file
file=open('C:\\Users\\Amy\\Downloads\\all1314text\\documentmatrix.txt','w')
file.write(str(news))
file.close()
#with open(outfname, "w") as outf:
#    outf.write(str(news))


In [2]:
with open("C:\\Users\\Amy\\Downloads\\all1314text\\documentmatrix.txt", "r") as document:
    news = ast.literal_eval(document.read())

In [3]:
news[0][3]

"UPDATE: As the Fiscal Cliff bill gets closer to likely passage, the Aussie market (one of the few markets that's trading right now) is at its highs of the day.  Hong Kong is up over 1.2%. EARLIER: There are very few markets open (Japan/China on Holiday, futures still closed) but one that's open is Australia. And so far the Washington theatrics are not a problem. \n"

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
actions=[]
for dayindex,daynews in enumerate(news):
    for articleindex,article in enumerate(daynews):
        
    
        action = {
        "_index": "article_index",
        "_type": "eachnews",
        "_id": f"{dayindex}-{articleindex}",
        "body":str(article)}
        actions.append(action)
#print (actions[:5])
# "body": " ".join(article)} 
helpers.bulk(es, actions)

(35898, [])

In [319]:
#test strings used later on
questionstring1= 'what % of drop in gdp is caused by investment'
questionstring2='who is the ceo of company Tesla'
questionstring3='who is the ceo of company Google'
questionstring4 = 'what impacts GDP'
questionstring5='which comapany went bankrupt in July of 2013?'
questionstring6="Which company went bankrupt in September 2008?"
questionstring7="which company declared bankruptcy in April 2014?"
questionstring8= 'what percentage of drop in gdp is associated with tax'
questionstring9= 'what percent of increase in gdp is associated with oil price'

# Question Classifier
we have two approaches to determine the question type, one is rule-based approach, another is automated classifer. Since rule-based approach is less tolerant with different wordings, I chose to compute the tf-idf score for each question as a term vector, and then get cosine similarity between this question vector with four questions and pick the highest similarity one. 

In [229]:
#need to classify questions
def classify_questions(questionstring):
    questioninreal = questionstring

    tfidf_vectorizer = TfidfVectorizer()
    #questioninreal = questioninreal.lower()
    tokenizedquestion=nltk.word_tokenize(questioninreal)


    question1 = "which companies went bankrupt in month x of year y?"
    question2 = "what affects gdp?"
    question3 = "what percentage % of drop or increase change in gdp is associated with results from this property consumption consumer spending government investment imports exports foreign trade?"
    question4 = "who is the ceo of company x?"

    questionlist=[]
    questionlist.append(question1)
    questionlist.append(question2)
    questionlist.append(question3)
    questionlist.append(question4)
    consinesim_list=[]
    for i in range(len(questionlist)):
        response=tfidf_vectorizer.fit_transform([questioninreal,questionlist[i]])
        cosin_sim=cosine_similarity(response)
        consinesim_list.append(cosin_sim)

    similaritylist=[]
    for item in consinesim_list:

        temporarymin=min(item[0])
        similaritylist.append(temporarymin)
    typeq= max(similaritylist)
    index=np.argmax(similaritylist)+1 #index in python is different
    return index

In [243]:
print(classify_questions(questionstring1))
print(classify_questions(questionstring2))
print(classify_questions(questionstring3))
print(classify_questions(questionstring4))
print(classify_questions(questionstring5))
print(classify_questions(questionstring6))
print(classify_questions(questionstring7))
print(classify_questions(questionstring8))
print(classify_questions(questionstring9))

3
4
4
2
1
1
1
3
3


# Extract keywords from the questions and formulate queries for es
get rid of stopwords
twist towards specific types of questions

In [7]:
from nltk.corpus import stopwords
stop_words_general = stopwords.words('english')

In [309]:
def processq(questionstring):
    newq=[]
    questionToken = nltk.word_tokenize(questionstring)
    if "?" in questionToken:
        questionToken.remove("?")
    for word in questionToken:
        if word.lower() not in stop_words_general:
            newq.append(word)
    #print (newq)
    return newq

test1=processq(questionstring1)
test2=processq(questionstring2)
test3=processq(questionstring3)
test4=processq(questionstring4)
test5=processq(questionstring5)
test6=processq(questionstring6)
test7=processq(questionstring7)
test8=processq(questionstring8)
test9=processq(questionstring9)

# Form a search query for different types of questions
## For ceo names
it's pretty easy in the way that the query only needs to contain "ceo" and the company name
a) use a subfunction called get_organization to get the name of the company
b) get the query for elastic search
## For queries about bankruptcy
use the same function get_organization to get the entity, but also need match years and months
## GDP query
"GDP" and keyverbs such as "affects,effect, affect, effects"
## Specific reason for GDP
"GDP"and that reason

In [302]:
from elasticsearch_dsl import Search, query
import re
def get_organization(textlist):
    ne_tree = nltk.ne_chunk(nltk.pos_tag(textlist))

    named_entities=[]
    for t in ne_tree.subtrees():
        if t.label() == 'PERSON' or t.label() == 'ORGANIZATION': #as organizations sometimes defined as person
            named_entities.append(" ".join([token for token, pos in t.leaves()]))
    return named_entities
 
def get_query_ceo(textlist):
    companyname=get_organization(textlist)
    companyname.append('CEO')
    #print(companyname)
    if len(companyname)==0:
        print("we can't recognize the company name")

    querystring= " AND ".join(companyname)
    queryq=query.Q("query_string",query=querystring)
    #print (queryq)
    return queryq

def get_query_company(textlist):
    #need to capture numeric year as well as Month
    monthdict={'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
    newmonth=[d for d in textlist if d in monthdict.keys()]
    monthval=newmonth[0]
    monthnum=monthdict[monthval]
    qyear=[]
    for item in textlist:
        year=re.findall(r"\d{4}",item)
        if len(year)>0:
            qyear.append(year)
    trueyear=int(qyear[0][0])
    #print (trueyear)
   
    q1 = query.Q("query_string", query = "bankruptcy bankrupt chapter filed declared" + " " +str(trueyear)+" "+str(monthval))
    return q1,str(trueyear),monthval

def get_query_gdp_reason(textlist):
    q1 = query.Q("query_string",query="GDP"+" "+"effects effect affect affects")
    #print (q1)
    return q1

def get_query_gdp_specific_percentage(textlist): #has to be in the list of q2's answer
    shouldnotincludelist=['%','percent','percentage','drop','gdp','associated','caused','increase']
    q2answerlist=['public debt', 'hurricane', 'credit', 'market', 'cost', 'investment', 'tax', 'deficit', 'oil', 'spend', 'debt']
    wordtaglist= nltk.pos_tag(textlist)
    askterm=[]
    for word,tag in wordtaglist:
        if tag=='NN':
            if word not in shouldnotincludelist:
                askterm.append(word)
    #print (askterm)
    
    askword=' '.join(askterm)
    #print (askword)

    q1 = query.Q("query_string", query = askword + " " +"GDP")
    #print (q1)
    return q1,askword


# Get documents searched result
Since we are using elasticsearch, the documents already ranked by similarities of keywords. So we chose top # of sentences by chosen. 

In [59]:
#def search(queryq):

def serach_from_query(queryq,chosensize):
    s=es.search(index='article_index',q=queryq,size=chosensize)

    candidates_docs=[]
    for i in range(len(s['hits']['hits'])):
        candidates_docs.append(s['hits']['hits'][i]['_source']['body'])

    #print (candidates_docs)
    return candidates_docs



# Answer-generated by potential documents
Rule based vs. Automated classifier

For CEO names, we use simple rule-based method as what we did similarly in hw3
select a subset of sentences that contain the word CEO and then choose the most common NER tag(using Spacy) that marks as Person(at least two words length) 

For Companies that go bankrupt, we select sentences if the sentence contains bankrupt/bankruptcy/chapter, and use spacy to filter if it's an organization or a government entity. Then select the most common word used.

For GDP, tf-idf is calculated for keywords and the phrase must both be nouns to be a reason. stopword dictionary is extended to delete uselss words

For GDP's reasons, regex is used to capture the percentage. Min val is returned. 


In [60]:
def get_answer_ceo(candidate_docs):
    candidatelistofsentences=[]
    for doc in candidate_docs:
        senttoken=sent_tokenize(doc)
        sent_ceo=filter(lambda x: "ceo" in x.lower(),senttoken)
        candidatelistofsentences.extend(sent_ceo)
    person_list=[]
    ceo_counts={}
    
    nlp=spacy.load('en_core_web_sm')
    for itemsent in candidatelistofsentences:
        doc=nlp(itemsent)
        for entity in doc.ents:
            if entity.label_ =='PERSON':
                if (len(entity.text.split()))>1:
                    person_list.append(entity.text)

    for item in person_list:
        occurence = map(lambda s: item in s, candidatelistofsentences)
        ceo_counts[item] = sum(occurence)
        ceo_counts=Counter(ceo_counts)
        
        result=ceo_counts.most_common(1)[0][0]
    #print (ceo_counts)
    return result


In [51]:
from spacy import displacy

def get_answer_company(candidate_docs,yearval,monthval):
    candidatelistofsentences=[]
    bankruptylist=['bankrupt','bankruptcy','chapter']
    for doc in candidate_docs:
        senttoken=sent_tokenize(doc) #list of sentences
        for everysent in senttoken:
            if any(word in everysent for word in bankruptylist):
                if monthval in everysent:
                    if yearval in everysent:
                        candidatelistofsentences.append(everysent)
    #print(candidatelistofsentences)
    company_list=[]
    company_counts={}
    nlp=spacy.load('en_core_web_sm')

    for itemsent in candidatelistofsentences:

        doc=nlp(itemsent)
    
        for entity in doc.ents:
            if entity.label_ =='ORG' or entity.label_=='GPE': #GOVERNMENT can go bankrupt
                company_list.append(entity.text)
    #print (company_list)
    
    new_company_list=[]
    for item in company_list:
        occurence = map(lambda s: item in s, candidatelistofsentences)
        company_counts[item] = sum(occurence)
        company_counts=Counter(company_counts)
        #print (company_counts)
        result=company_counts.most_common(1)[0][0]
    return result             

In [223]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

def get_answer_gdp_reason(candidate_docs):
    candidatelistofsentences=[]
    for doc in candidate_docs:
        senttoken=sent_tokenize(doc) #list of sentences
        for everysent in senttoken:
            if "GDP" in everysent:
                candidatelistofsentences.append(everysent)
    #print (candidatelistofsentences)
    #print (len(candidatelistofsentences))
    
    lmtzr = WordNetLemmatizer()
    lemmatized = [[lmtzr.lemmatize(word,'v') for word in word_tokenize(s)]
                  for s in candidatelistofsentences]


    cleaned_lemmatizedlist=[]
    for everylist in lemmatized:
        #print (everylist)
        newtrial=' '.join(['%s' % w for w in everylist])
        cleaned_lemmatizedlist.append(newtrial)

    #print (cleaned_lemmatizedlist)    
    tf = TfidfVectorizer(ngram_range = (1, 2),stop_words=stopwordslist)
    tfidf_matrix = tf.fit_transform(cleaned_lemmatizedlist)
    feature_names=tf.get_feature_names()
    
    term_freqs = np.sum(tfidf_matrix, axis = 0)
    inds = np.argsort(term_freqs)[:, -100:]
    words = np.array(tf.get_feature_names())[inds]

    #print (words)

    flat_word_list = [item for sublist in words for item in sublist]

    testnnword=[]
    testpostag=nltk.tag.pos_tag(flat_word_list)
    for word,tag in testpostag:

       # print (word,tag)
        if tag =='NN'or tag =='NNS' or tag=='NNP':

            if word not in stopwordslist:
                testnnword.append(word)
    #print (testnnword)
    newstopwords=['country','result','data','project','forecast','rate','china','term','world','growth','change','drop','figure','level','fall','report','impact','see','grow','rise','economy','rise','years','cbo','price','government','increase','point','quarter','year','countries','decline']

    reasons=[]
    for longword in testnnword:
        if any(word in longword for word in newstopwords):
            continue
        else:
            reasons.append(longword)
    #print (reasons)  
    return reasons

In [310]:
def get_answer_gdp_pct(candidate_docs,askwordq):
    candidatelistofsentences=[]
    for doc in candidate_docs:
        senttoken=sent_tokenize(doc) #list of sentences
        for everysent in senttoken:
            if "GDP" in everysent:
                if askwordq in everysent:
                    candidatelistofsentences.append(everysent)
    #print (candidatelistofsentences)
    #print (len(candidatelistofsentences))
    
    percents = [ ]
    pattern = re.compile("\d+(?:\.\d+)?(?:%| percent(?:(?:age|ile)? points)?)")
    for sentence in candidatelistofsentences:
        matches = re.findall(pattern, sentence)
        percents.extend(matches)
    #print (percents)

    numericvallist=[]
    pattern2=re.compile("\d+(?:\.\d+)?")
    for percent in percents:
        matchespct = re.findall(pattern2, percent)
        numericvallist.extend(matchespct)
    
    #print (min(numericvallist))
    answerstring = min(numericvallist)+'%'
    #print (answerstring)
    return answerstring

## Warning:
The following two cells is for reference of how to get gdp's reason. I used a formal version of it as a function, but it is left here as referencing the above result I got 

In [227]:
def get_answer_gdp_reason_test(candidate_docs):
    candidatelistofsentences=[]
    for doc in candidate_docs:
        senttoken=sent_tokenize(doc) #list of sentences
        for everysent in senttoken:
            if "GDP" in everysent:
                candidatelistofsentences.append(everysent)
    #print (candidatelistofsentences)
    print (len(candidatelistofsentences))
    return candidatelistofsentences

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
sentsgdp=get_answer_gdp_reason_test(gdpdoc)
#print (sentsgdp)

from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

lmtzr = WordNetLemmatizer()
lemmatized = [[lmtzr.lemmatize(word,'v') for word in word_tokenize(s)]
              for s in sentsgdp]


cleaned_lemmatizedlist=[]
for everylist in lemmatized:
    #print (everylist)
    newtrial=' '.join(['%s' % w for w in everylist])
    cleaned_lemmatizedlist.append(newtrial)
    
#print (cleaned_lemmatizedlist)    
tf = TfidfVectorizer(ngram_range = (1, 2),stop_words=stopwordslist)
tfidf_matrix = tf.fit_transform(cleaned_lemmatizedlist)
feature_names=tf.get_feature_names()

term_freqs = np.sum(tfidf_matrix, axis = 0)
inds = np.argsort(term_freqs)[:, -100:]
words = np.array(tf.get_feature_names())[inds]

print (words)

flat_word_list = [item for sublist in words for item in sublist]

testnnword=[]
testpostag=nltk.tag.pos_tag(flat_word_list)
for word,tag in testpostag:
    
   # print (word,tag)
    if tag =='NN'or tag =='NNS' or tag=='NNP':

        if word not in stopwordslist:
            testnnword.append(word)
print (testnnword)
newstopwords=['country','result','data','project','forecast','rate','china','term','world','growth','change','drop','figure','level','fall','report','impact','see','grow','rise','economy','rise','years','cbo','price','government','increase','point','quarter','year','countries','decline']

reasons=[]
for longword in testnnword:
    if any(word in longword for word in newstopwords):
        continue
    else:
        reasons.append(longword)
print (reasons)  


244
[['public debt' 'private' 'hurricane' 'come' 'actual' 'trillion' 'shrink'
  'credit' '30' 'however' '2023' 'continue' 'second' 'higher'
  'government debt' 'country' 'q1' 'data' 'reduce' 'chart' 'china'
  'remain' '100' 'also' 'end' 'result' 'market' 'project' 'per'
  'forecast' 'rate' 'world' 'around' 'total' 'real growth' '2012' '2008'
  'annual' '2007' 'change' 'drop' 'term' 'federal' 'cost' 'since'
  'public' 'figure' 'fed' 'expect' 'global' 'economic growth' 'lower'
  'potential' 'cbo' 'billion' 'negative' 'level' 'find' 'investment'
  'first' 'affect' 'fall' 'report' 'oil price' 'much' 'impact' 'could'
  'tax' 'average growth' 'see' 'deficit' 'add' 'grow' 'say' 'current'
  'decline' 'rise' '2013' '90' 'us' '2014' 'countries' 'show' 'economy'
  'years' 'price' 'government' 'increase' 'oil' 'point' 'spend' 'real'
  'economic' 'quarter' 'would' 'average' 'year' 'estimate' 'debt'
  'growth']]
['public debt', 'hurricane', 'credit', 'government debt', 'country', 'data', 'china', 'r

In [304]:
querytestgdpperct,askwordq=get_query_gdp_specific_percentage(test9)
perctdoc=serach_from_query(querytestgdpperct,50)
get_answer_gdp_pct(perctdoc,askwordq)

['1.6%', '4.8%', '40%', '1 percent', '10%', '0.2%', '20%', '1%', '1.6%', '4.8%', '1 percent', '0.4 percentage points', '0.2 percent', '35%', '90%', '20%', '0.5%', '1.3%', '9%', '12.7%', '10%', '4.7%', '8.0%', '20%', '0.5%', '1.3%', '9%', '12.7%', '10%', '4.7%', '8.0%', '1.6%', '1.6%', '1.5%', '1.5%', '0.2%', '0.5%']


'0.2%'

In [311]:
def answer_the_question(questionstring):
    typeq= classify_questions(questionstring)
    questiontextlist=processq(questionstring)
    if typeq==1:
        company_query,yearval,monthval=get_query_company(questiontextlist)
        candidate_doc=serach_from_query(company_query,1000)
        names=get_answer_company(candidate_doc,yearval,yearval)
    if typeq==2:
        querytestgdp=get_query_gdp_reason(questiontextlist)
        gdpdoc=serach_from_query(querytestgdp,100)
        names=get_answer_gdp_reason(gdpdoc)
    if typeq==3:
        querytestgdpperct,askwordq=get_query_gdp_specific_percentage(questiontextlist)
        perctdoc=serach_from_query(querytestgdpperct,10)
        names=get_answer_gdp_pct(perctdoc,askwordq)
    if typeq==4:
        ceo_query=get_query_ceo(questiontextlist)
        candidate_doc=serach_from_query(ceo_query,50)
        names=get_answer_ceo(candidate_doc) #tesla - elon musk
    return names

In [312]:
print(answer_the_question("what impacts GDP"))
print(answer_the_question("what affects GDP"))
print(answer_the_question("what causes GDP"))

['public debt', 'hurricane', 'credit', 'market', 'cost', 'investment', 'tax', 'deficit', 'oil', 'spend', 'debt']
['public debt', 'hurricane', 'credit', 'market', 'cost', 'investment', 'tax', 'deficit', 'oil', 'spend', 'debt']
['public debt', 'hurricane', 'credit', 'market', 'cost', 'investment', 'tax', 'deficit', 'oil', 'spend', 'debt']


In [313]:
print(answer_the_question("what percent of increase in gdp is associated with oil price"))
print(answer_the_question("what percentage of drop in gdp is associated with tax"))
print(answer_the_question("what % of drop in gdp is caused by investment"))

0.2%
0.8%
1.18%


In [317]:
print (answer_the_question('who is the CEO of company Google?'))
print(answer_the_question('who is the CEO of Facebook'))
print(answer_the_question('who is the ceo of the company IBM'))
print(answer_the_question('who is the ceo of the company Morgan Stanley'))

Larry Page
Mark Zuckerberg
Ginni Rometty
James Gorman


In [318]:
print(answer_the_question('which comapany went bankrupt in July of 2013?'))
print(answer_the_question("Which company went bankrupt in September 2008?"))
print(answer_the_question("which company declared bankruptcy in June 2009?"))

Detroit
Lehman
GM
